In [ ]:
import Data.Bits             (Bits (bit, complement, popCount, shiftR, (.&.), (.|.)),
                              FiniteBits (finiteBitSize))
import Data.ByteArray.Hash   (FnvHash32 (..), fnv1Hash)
import Data.ByteString.Char8 (pack)
import Data.Char             (intToDigit)
import Data.Semigroup        ((<>))
import Data.Vector           (Vector, drop, singleton, take, (!), (//))
import Data.Word             (Word16, Word32)
import Numeric               (showIntAtBase)
import Prelude               hiding (drop, lookup, take)
import System.TimeIt         (timeIt)
import Text.Show.Pretty      (pPrint)

In [ ]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

newtype Binary a = Binary a
    deriving (Enum, Ord, Real, Integral, Eq, Num, Bits, FiniteBits)

instance (FiniteBits a, Show a, Integral a) => Show (Binary a) where
    show (Binary n) = let
        str = showIntAtBase 2 intToDigit n ""
        size = finiteBitSize $ undefined `asTypeOf` n
        in replicate (size - length str) '0' <> str

In [ ]:
24732 :: Word16

In [ ]:
24732 :: Binary Word16

In [ ]:
type Hash = Binary Word32
type Bitmap = Binary Word16

In [ ]:
bitsPerSubkey :: Int
bitsPerSubkey = 4

In [ ]:
type Shift = Int

In [ ]:
class Hashable a where
    hash :: a -> Hash

In [ ]:
{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances #-}

instance Hashable String where
    hash s = let
        FnvHash32 h = fnv1Hash (pack s)
        in Binary h

In [ ]:
hash "1" :: Binary Word32

In [ ]:
data HAMT key value
    = None
    | Leaf Hash key value
    | Many Bitmap (Vector (HAMT key value))
    deriving (Show)

empty :: HAMT k v
empty = None

In [ ]:
insertAt :: Vector a -> Int -> a -> Vector a
insertAt vector index a = take index vector <> singleton a <> drop index vector

updateAt :: Vector a -> Int -> a -> Vector a
updateAt vector index a = vector // [(index, a)]

deleteAt :: Vector a -> Int -> Vector a
deleteAt vector index = take index vector <> drop (index+1) vector

In [ ]:
h = hash "1"
leaf = Leaf h "1" 1

leaf

In [ ]:
subkeyMask :: Bitmap
subkeyMask = (bit bitsPerSubkey) - 1

subkeyMask

In [ ]:
--     0101110100101110
-- .&. 0000000000001111
-----------------------
--     0000000000001110

fragment = fromIntegral h .&. subkeyMask

fragment

In [ ]:
Binary position = fragment

position

In [ ]:
someBitmap :: Bitmap
someBitmap = Binary $ bit $ fromIntegral position

someBitmap

In [ ]:
bitMask_ :: Hash -> Bitmap
bitMask_ hash = let
    fragment = fromIntegral hash .&. subkeyMask
    Binary position = fragment
    in Binary (bit (fromIntegral position))

In [ ]:
mask = bitMask_ (hash "10")

mask

In [ ]:
mask - 1

In [ ]:
--     0100000000000000
-- .&. 0000001111111111
-----------------------
--     0000000000000000

masked = someBitmap .&. (mask - 1)

masked

In [ ]:
index = popCount masked

index

In [ ]:
maskIndex :: Bitmap -> Bitmap -> Int
maskIndex bitmap mask = popCount (bitmap .&. (mask - 1))

In [ ]:
subkeyMask :: Bitmap
subkeyMask = (bit bitsPerSubkey) - 1

maskIndex :: Bitmap -> Bitmap -> Int
maskIndex bitmap mask = popCount (bitmap .&. (mask - 1))

subkey :: Hash -> Shift -> Int
subkey hash shift = fromIntegral $ (fromIntegral $ shiftR hash shift) .&. subkeyMask

bitMask :: Hash -> Shift -> Bitmap
bitMask hash shift = bit (subkey hash shift)

In [ ]:
insert :: Hashable key => key -> value -> HAMT key value -> HAMT key value
insert key value hamt = insert' 0 (hash key) key value hamt

insert' :: Shift -> Hash -> key -> value -> HAMT key value -> HAMT key value
insert' shift hash key value None = Leaf hash key value

insert' shift hash key value leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = Leaf hash key value
    | otherwise = insert' shift hash key value (Many (bitMask leafHash shift) (singleton leaf))

insert' shift hash key value (Many bitmap vector)
    | bitmap .&. mask == 0 = let
        leaf = Leaf hash key value
        vector' = insertAt vector index leaf
        bitmap' = bitmap .|. mask
        in Many bitmap' vector'
    | otherwise = let
        subtree = vector ! index
        subtree' = insert' (shift+bitsPerSubkey) hash key value subtree
        vector' = updateAt vector index subtree'
        in Many bitmap vector'
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [ ]:
fromList :: Hashable key => [(key, value)] -> HAMT key value
fromList = foldr (uncurry insert) empty

example = fromList [("1", 1), ("10", 2), ("100", 3), ("1000", 4)]

pPrint example

In [ ]:
lookup :: Hashable key => key -> HAMT key value -> Maybe value
lookup key hamt = lookup' 0 (hash key) hamt

lookup' :: Shift -> Hash -> HAMT key value -> Maybe value
lookup' shift hash None = Nothing

lookup' shift hash (Leaf leafHash leafKey leafValue)
    | hash == leafHash = Just leafValue
    | otherwise = Nothing

lookup' shift hash (Many bitmap vector)
    | bitmap .&. mask == 0 = Nothing
    | otherwise = lookup' (shift+bitsPerSubkey) hash (vector ! index)
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [ ]:
lookup "100" example

In [ ]:
fib :: Int -> Int
fib 0 = 1
fib 1 = 1
fib n = fib (n-1) + fib (n-2)

timeIt $ print $ fib 30

In [ ]:
instance Hashable Int where
    hash int = Binary (fromIntegral int)

fib' :: HAMT Int Int -> Int -> (Int, HAMT Int Int)
fib' table 0 = (1, insert 0 1 table)
fib' table 1 = (1, insert 1 1 table)
fib' table n = case lookup n table of
    Just i -> (i, table)
    Nothing -> let
        (i1, table')  = fib' table  (n-1)
        (i2, table'') = fib' table' (n-2)
        in (i1 + i2, insert n (i1 + i2) table'')

fib :: Int -> Int
fib n = fst $ fib' empty n

timeIt $ print $ fib 30

In [ ]:
delete :: Hashable key => key -> HAMT key value -> HAMT key value
delete key hamt = delete' 0 (hash key) hamt

delete' :: Shift -> Hash -> HAMT key value -> HAMT key value
delete' shift hash None = None

delete' shift hash leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = None
    | otherwise = leaf

delete' shift hash many@(Many bitmap vector)
    | bitmap .&. mask == 0 = many
    | otherwise = let
        subtree = vector ! index
        subtree' = delete' (shift+bitsPerSubkey) hash subtree
        in case subtree' of
            None -> if length vector == 1
                then None
                else Many (bitmap .&. complement mask) (deleteAt vector index)
            Leaf{} -> if length vector == 1
                then subtree'
                else  Many bitmap (updateAt vector index subtree')
            Many{} -> Many bitmap (updateAt vector index subtree')
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [ ]:
pPrint $ delete "1000" example

In [ ]:
pPrint $ delete "10" $ delete "1000" example